In [ ]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_lg")


In [ ]:
from textblob import TextBlob


In [ ]:
blob = TextBlob(text)
blob.noun_phrases

In [ ]:
description = "Sets the pivot point for scale operations."
method = "void setPivotX(float pivotX)"
class_ = "android.support.constraint.helper.Layer"

In [ ]:
def extract_method_name(text):
    method_name = text.split("(")[0].split()[1]
    parts = []
    substr = ""
    for c in method_name:
        if c.isupper():
            if substr:
                parts.append(substr.lower())            
            substr = c
        else:
            substr += c
    parts.append(substr.lower())    
    return parts

def extract_class_name(text):
    class_name = text.split(".")[-1]
    parts = []
    substr = ""
    for c in class_name:
        if c.isupper():
            if substr:
                parts.append(substr.lower())
            substr = c
        else:
            substr += c
    parts.append(substr.lower())    
    return parts

def extract_dobj(text):
    doc = nlp(text)
    for token in doc:
        if token.dep_ == "dobj":
            return token.text.lower()
        



In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
def remove_stopwords(word_list):
    filtered_sentence = [w for w in word_list if not w in stop_words] 
    return filtered_sentence

In [ ]:
raw_method_name = extract_method_name(method)
filtered_method_name = remove_stopwords(raw_method_name[1:])
method_name = " ".join(filtered_method_name)

In [ ]:
raw_class_name = extract_class_name(class_)
class_name = " ".join(raw_class_name)

In [ ]:
class_name

In [ ]:
method_name

In [ ]:
filtered_method_name

In [ ]:
extract_dobj(description)

In [ ]:
import os
import numpy as np

from gensim.models import KeyedVectors
from nltk import word_tokenize


In [ ]:
def load_embeddings_file(file_name, embedding_type, lower):
    """TODO"""
    if not os.path.isfile(file_name):
        raise Exception("{} does not exist".format(file_name))
    words = None
    if embedding_type == "word2vec":
        model = KeyedVectors.load_word2vec_format(
            file_name, binary=True, unicode_errors="ignore"
        )
        words = model.index2entity
    else:
        raise Exception("Unknown Embedding Type")

    if lower:
        vectors = {word.lower(): model[word] for word in words}
    else:
        vectors = {word: model[word] for word in words}
    return vectors


In [ ]:
embeddings = load_embeddings_file("../../datasets/embeddings/GoogleNews-vectors-negative300.bin.gz", "word2vec", True)



In [ ]:
from nltk.tokenize import word_tokenize

def sentence_embeddings(sentence, embeddings):
    s_vectors = [embeddings[word] for word in word_tokenize(sentence) if word in embeddings ]
    embedding = np.sum(s_vectors, axis=0)
    return embedding

In [ ]:
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
def algorithm(description, method, class_, embeddings):
    obj = extract_dobj(description)
    
    raw_method_name = extract_method_name(method)
    filtered_method_name = remove_stopwords(raw_method_name[1:])
    method_name = " ".join(filtered_method_name)
    
    raw_class_name = extract_class_name(class_)
    class_name = " ".join(raw_class_name)
    
    name = ""
    if method_name:
        name = method_name
    else:
        name = class_name
    
    if name:
        obj_embedding = sentence_embeddings(obj, embeddings)
        method_embedding = sentence_embeddings(name, embeddings)
        
        similarity = dot(obj_embedding.tolist(), method_embedding.tolist())/(norm(obj_embedding.tolist())*norm(method_embedding.tolist()))
        print("{}:{}:{}".format(obj, name, similarity))
    
    
        

In [ ]:
#algorithm(description, method, class_, embeddings)

In [1]:
import spacy
from benepar.spacy_plugin import BeneparComponent


/Users/huseyinalecakir/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/huseyinalecakir/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/huseyinalecakir/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/huseyinalecakir/anaconda3/lib/python3.6/site-

In [3]:
nlp = spacy.load("en_core_web_lg")


In [4]:
nlp.add_pipe(BeneparComponent('benepar_en'))


In [6]:
doc = nlp("The time for action is now. It's never too late to do something.")


In [7]:
doc.sents

In [11]:
sents = list(doc.sents)

In [12]:
sent = sents[0]

In [13]:
print(sent._.parse_string)

(S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))


In [14]:
print(sent._.labels)

('S',)


In [20]:
list(sent._.children)[0]

The time for action

In [ ]:
print(list(sent._.children)[0])

In [24]:
import benepar
parser = benepar.Parser("benepar_en2")
tree = parser.parse("Short cuts make long delays.")

In [25]:
tree.pretty_print()

                S                      
        ________|____________________   
       |             VP              | 
       |         ____|____           |  
       NP       |         NP         | 
   ____|___     |     ____|____      |  
  JJ      NNS  VBP   JJ       NNS    . 
  |        |    |    |         |     |  
Short     cuts make long     delays  . 



In [38]:
def extract_vp(parent):
    for node in parent:
        if type(node) is nltk.Tree:
            label = node.label()
            if label == "VP":
                surface = " ".join(node.leaves())
                return surface

In [39]:
extract_vp(tree)

'make long delays'